In [ ]:
import requests
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time, datetime
import gzip
import shutil
import csv
import seaborn as sns

In [ ]:
url = "https://api.coingecko.com/api/v3/coins/dogecoin/market_chart/range"

params = {
    "vs_currency": "usd",
    "from": 1728489600,   # 2024-10-10
    "to": 1736092800      # 2025-01-06
}

# Fetch data
response = requests.get(url, params = params)
data = response.json()
print(data.keys())

In [ ]:
df_p = pd.DataFrame(data["prices"], columns = ["timestamp", "price"])
df_v = pd.DataFrame(data["total_volumes"], columns = ["timestamp", "volume"])
df_pv = pd.merge(df_p , df_v, on = "timestamp", how = "inner")

df_pv["timestamp"] = pd.to_datetime(pd.to_numeric(df_pv["timestamp"]), unit = "ms")
df_pv["price"] = df_pv["price"].astype(float)
df_pv["volume"] = df_pv["volume"].astype(float)

print(df_pv.head(5))
print(df_pv.shape) # (2111, 3)

In [ ]:
first_date = min(df_pv["timestamp"]).date() 
last_date = max(df_pv["timestamp"]).date() 

y1 = df_pv["volume"] 
y2 = df_pv["price"] 
x = df_pv["timestamp"] 

fig, ax1 = plt.subplots(figsize = (8,6)) 

# Plot barplot for volume on primary y-axis 
ax1.bar(x, y1, color = "steelblue", label = "Volume") 
ax1.set_xlabel("Date (Month-Day)") 
ax1.set_ylabel("Volume", color = "steelblue") 
ax1.tick_params(axis = "y", labelcolor = "steelblue") 

# Plot lineplot for price on secondary y-axis 
ax2 = ax1.twinx() 
ax2.plot(x, y2, color = "indianred", alpha = 0.8, label = "Price") 
ax2.set_ylabel("Price", color = "indianred") 
ax2.tick_params(axis = "y", labelcolor = "indianred") 

annotation = df_pv.nlargest(1, "volume")[["timestamp", "volume"]]
ax1.annotate(
        f"volume spike: \n {int(annotation["volume"]):,}",
        xy = (annotation["timestamp"], annotation["volume"]), xycoords = "data",
        xytext = (-60, -10), textcoords = "offset points", fontsize = 8,
        ha = "center", va = "bottom",
        arrowprops = dict(facecolor = "black", shrink = 0.01, width = 0.5, headwidth = 4),
        bbox = dict(facecolor = "white", alpha = 0.7, edgecolor = "black", boxstyle = "round,pad=0.2")
        )    

# Create legends 
legend1 = ax1.legend(loc = "upper left") 
legend2 = ax2.legend(loc = "upper right") 

# Format and show 
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d')) 
fig.autofmt_xdate() 

plt.suptitle("Dogecoin Hourly Price and Volume", y = 1, fontsize = 15) 
plt.title(f"{first_date} to {last_date}", fontsize = 12) 
fig.tight_layout() # Adjust layout to prevent labels from overlapping 
plt.show()

Decide to zoom in to week of 9 nov to 15 nov 2024

In [ ]:
target_cols = ["time", "input_count", "output_count", "output_total_usd", "cdd_total"]

list_of_dfs = []

for i in range(9, 16):  # Week of 9-15 Nov 2025
    date = f"202411{i:02d}"  # Ensures 2-digit day in string
    url = f"https://gz.blockchair.com/dogecoin/transactions/blockchair_dogecoin_transactions_{date}.tsv.gz"
    filename = f"blockchair_dogecoin_transactions_{date}.tsv.gz"

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

    with gzip.open(filename, "rb") as f_in:
        with open(filename.replace(".gz", ""), "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

    # Read in chunks for efficiency
    chunks = pd.read_csv(
        filename.replace(".gz", ""), 
        sep = "\t", 
        usecols = target_cols, 
        chunksize = 100000
    )
    
    # Convert column types inside each chunk
    for chunk in chunks: 
        chunk = chunk.astype({
            "input_count": "int32",
            "output_count": "int32",
            "output_total_usd": "int32",
            "cdd_total": "float32"
        })

        # Round timestamp to nearest hour to match df_pv timestamps
        chunk["time"] = pd.to_datetime(chunk["time"]).dt.round("h")

        # Group by hour and compute sum across all transactions in that hour
        chunk_grouped = chunk.groupby("time").sum(numeric_only = True)
        list_of_dfs.append(chunk_grouped)

# Concatenate dataframes across all 7 days
df_tx_hourly = pd.concat(list_of_dfs).groupby("time").sum().reset_index()

# Save as CSV file
df_tx_hourly.to_csv("df_tx_hourly.csv", index = False)

In [ ]:
# do work on df_pv
df_pv = df_pv[(df_pv["timestamp"] >= pd.to_datetime("2024-11-09")) & 
              (df_pv["timestamp"] < pd.to_datetime("2024-11-16"))]
df_pv["timestamp"] = df_pv["timestamp"].dt.round("h")
df_pv.rename(columns = {"timestamp": "time"}, inplace = True)
df_pv.to_csv("df_pv.csv", index = False) # save as csv
print(df_pv.shape) # (168, 3)
print(df_pv)

In [ ]:
DF = pd.merge(df_tx_hourly , df_pv, on = "time", how = "left")
DF.to_csv("DF.csv", index = False) # save as csv
print(DF.shape) # (169, 7)

In [ ]:
high_output_total = DF.nlargest(3, "output_total_usd")[["time", "volume", "output_total_usd"]]

first_date = min(DF["time"]).date() 
last_date = max(DF["time"]).date() 

y1 = DF["volume"] 
y2 = DF["output_total_usd"] 
x = DF["time"] 

fig, ax1 = plt.subplots(figsize = (7,6)) 

# Plot barplot for volume on primary y-axis 
ax1.bar(x, y1, color = "lightblue", label = "Volume") 
ax1.set_xlabel("Date (Month-Day)") 
ax1.set_ylabel("Volume", color = "steelblue") 
ax1.tick_params(axis = "y", labelcolor = "steelblue") 

# Plot lineplot for on-chain transaction flow on secondary y-axis 
ax2 = ax1.twinx() 
ax2.plot(x, y2, color = "indianred", label = "Total On-chain \n Output (USD)") 
ax2.set_ylabel("Total On-chain Output (USD)", color = "indianred") 
ax2.tick_params(axis = "y", labelcolor = "indianred") 

# Create annotations for 2 highest hourly outputs
highest = high_output_total.iloc[0]
ax2.annotate(
        f"Highest output: \n {int(highest["output_total_usd"]):,}, \n Time: {highest["time"]}",
        xy = (highest["time"], highest["output_total_usd"]), xycoords = "data",
        xytext = (80, -50), textcoords = "offset points", fontsize = 8,
        ha = "center", va = "bottom",
        arrowprops = dict(facecolor = "black", shrink = 0.01, width = 0.5, headwidth = 4),
        bbox = dict(facecolor = "white", alpha = 0.7, edgecolor = "black", boxstyle = "round,pad=0.2")
        )

highest2 = high_output_total.iloc[1]
ax2.annotate(
        f"Second highest output: \n {int(highest2["output_total_usd"]):,} \n Time: {highest2["time"]}",
        xy = (highest2["time"], highest2["output_total_usd"]), xycoords = "data",
        xytext = (-80, -30), textcoords = "offset points", fontsize = 8,
        ha = "center", va = "bottom",
        arrowprops = dict(facecolor = "black", shrink = 0.01, width = 0.5, headwidth = 4),
        bbox = dict(facecolor = "white", alpha = 0.7, edgecolor = "black", boxstyle = "round,pad=0.2")
        )   

highest3 = high_output_total.iloc[2]
ax2.annotate(
        f"Third highest output: \n {int(highest3["output_total_usd"]):,} \n Time: {highest3["time"]}",
        xy = (highest3["time"], highest3["output_total_usd"]), xycoords = "data",
        xytext = (80, -70), textcoords = "offset points", fontsize = 8,
        ha = "center", va = "bottom",
        arrowprops = dict(facecolor = "black", shrink = 0.01, width = 0.5, headwidth = 4),
        bbox = dict(facecolor = "white", alpha = 0.7, edgecolor = "black", boxstyle = "round,pad=0.2")
        )        

# Create legends 
legend1 = ax1.legend(loc = "upper left") 
legend2 = ax2.legend(loc = "upper right") 

# Format and show 
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d')) 
fig.autofmt_xdate() 

plt.suptitle("Dogecoin Hourly Volume and On-chain Transaction Flow", y = 1, fontsize = 15) 
plt.title(f"{first_date} to {last_date}", fontsize = 12) 
fig.tight_layout()  
plt.show()

In [ ]:
high_cdd_total = DF.nlargest(3, "cdd_total")[["time", "volume", "cdd_total"]]

first_date = min(DF["time"]).date() 
last_date = max(DF["time"]).date() 

y1 = DF["volume"] 
y2 = DF["cdd_total"] 
x = DF["time"] 

fig, ax1 = plt.subplots(figsize = (7,6)) 

# Plot barplot for volume on primary y-axis 
ax1.bar(x, y1, color = "lightblue", label = "Volume") 
ax1.set_xlabel("Date (Month-Day)") 
ax1.set_ylabel("Volume", color = "steelblue") 
ax1.tick_params(axis = "y", labelcolor = "steelblue") 

# Plot lineplot for on-chain CDD on secondary y-axis 
ax2 = ax1.twinx() 
ax2.plot(x, y2, color = "forestgreen", label = "Total CDD") 
ax2.set_ylabel("Total CDD", color = "forestgreen") 
ax2.tick_params(axis = "y", labelcolor = "forestgreen") 

# Create annotations for highest hourly CDD and range
highest = high_cdd_total.iloc[0]
ax2.annotate(
        f"Highest CDD: \n {int(highest["cdd_total"]):,}, \n Time: {highest["time"]}",
        xy = (highest["time"], highest["cdd_total"]), xycoords = "data",
        xytext = (80, -50), textcoords = "offset points", fontsize = 8,
        ha = "center", va = "bottom",
        arrowprops = dict(facecolor = "black", shrink = 0.01, width = 0.5, headwidth = 4),
        bbox = dict(facecolor = "white", alpha = 0.7, edgecolor = "black", boxstyle = "round,pad=0.2")
        )  

ax2.annotate(
        f"Range of high CDD \n Time: {high_cdd_total.iloc[1]["time"]} \n to {high_cdd_total.iloc[2]["time"]}", 
        xy = (0.445, 0.6), xycoords = "axes fraction", 
        xytext = (0.445, 0.65), fontsize = 8, 
        ha = "center", va = "bottom",
        arrowprops = dict(arrowstyle = "-[, widthB=5.0, lengthB=1.5", lw = 1.5, color = "black"),
        bbox = dict(facecolor = "white", alpha = 0.7, edgecolor = "black", boxstyle = "round,pad=0.2")
        )

# Create legends 
legend1 = ax1.legend(loc = "upper left") 
legend2 = ax2.legend(loc = "upper right") 

# Format and show 
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d')) 
fig.autofmt_xdate() 

plt.suptitle("Dogecoin Hourly Volume and Total CDD", y = 1, fontsize = 15) 
plt.title(f"{first_date} to {last_date}", fontsize = 12) 
fig.tight_layout()
plt.show()

In [ ]:
y1 = DF["output_total_usd"] 
y2 = DF["cdd_total"] 
x = DF["time"] 

fig, ax1 = plt.subplots(figsize = (7,6)) 

# Plot lineplot for on-chain transaction flow on primary y-axis 
ax1.plot(x, y1, color = "indianred", alpha = 0.7, label = "Total On-chain \n Output (USD)") 
ax1.set_xlabel("Date (Month-Day)") 
ax1.set_ylabel("Total On-chain Output (USD)", color = "indianred") 
ax1.tick_params(axis = "y", labelcolor = "indianred") 

# Plot lineplot for on-chain CDD on secondary y-axis 
ax2 = ax1.twinx() 
ax2.plot(x, y2, color = "forestgreen", alpha = 0.7, label = "Total CDD") 
ax2.set_ylabel("Total CDD", color = "forestgreen") 
ax2.tick_params(axis = "y", labelcolor = "forestgreen") 

# Create legends 
legend1 = ax1.legend(loc = "upper left") 
legend2 = ax2.legend(loc = "upper right") 

# Format and show 
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d')) 
fig.autofmt_xdate() 

plt.suptitle("Dogecoin Hourly On-chain Transaction Flow and Total CDD", y = 1, fontsize = 15) 
plt.title(f"{first_date} to {last_date}", fontsize = 12) 
fig.tight_layout()
plt.show()

Period of interest 1:
output spike 11/11 4pm, around high CDD range 11/11 9am to 11/12 5pm
Period of interest 2:
output spike 11/12 3pm and 5pm, followed by highest CDD spike that night 11/13 4am 

In [ ]:
target_cols = ["time", "input_count", "output_count", "output_total_usd", "cdd_total"]

dates = ["20241111", "20241112", "20241113"] # 11-13 Nov

for date in dates:
    url = f"https://gz.blockchair.com/dogecoin/transactions/blockchair_dogecoin_transactions_{date}.tsv.gz"
    filename = f"blockchair_dogecoin_transactions_{date}.tsv.gz"

    # Download compressed file
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

    # Unzip
    with gzip.open(filename, "rb") as f_in:
        with open(filename.replace(".gz", ""), "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

    # Read in chunks for efficiency
    chunks = pd.read_csv(
        filename.replace(".gz", ""), 
        sep = "\t", 
        usecols = target_cols, 
        chunksize = 100000
    )

    # Convert column types inside each chunk
    list_of_chunks = []
    for chunk in chunks:
        chunk = chunk.astype({
            "input_count": "int32",
            "output_count": "int32",
            "output_total_usd": "int32",
            "cdd_total": "float32"
        })
        list_of_chunks.append(chunk)

    # Concatenate chunks into one dataframe named as date
    df_day = pd.concat(list_of_chunks, ignore_index = True)

    # Save as CSV file
    df_day.to_csv(f"df_{date}_hourly.csv", index = False)

input_count → Number of inputs (wallets sending DOGE into this transaction).

output_count → Number of outputs (wallets receiving DOGE).

input_total → Total DOGE amount coming from inputs, in smallest unit (1 DOGE = 10⁸ satoshis).

output_total → Total DOGE amount going to outputs (sum of all outputs).

input_total_usd / output_total_usd → Approximate USD value at the time of the transaction.


input_count & output_count → Many outputs in a single tx might be a dispersal, mixing, or pump-and-dump distribution.
fee_per_kb → Extremely low fees may indicate spam or batch transactions; very high fees could signal priority/whale moves.
cdd_total → Moving old coins may indicate a whale exiting a position.
time + volume → Align with price spikes to detect suspicious activity

1. Large transactions:
Sort output_total descending → top 5–10 transactions may be whales.
2. Many recipients:
Sort output_count descending → txs sending to many wallets could indicate dispersal.
3. High CDD:
Sort cdd_total descending → old coins moving.
4. Combine metrics:
output_total + output_count + cdd_total → transactions that are large, spread out, and involve old coins → classic pump-and-dump / suspicious move.


Big on-chain transfers near price/volume spikes.
Sudden bursts of activity from new wallets or concentrated addresses.
Wallets that fan out DOGE to many addresses (classic pump-and-dump prep).
New wallets suddenly transacting.

1. Time series with highlighted anomalies 📈
Best for: spotting pump-and-dump schemes or wash trading.
Why: sudden spikes in price or volume are the most common red flag.
You can overlay:
Price (line plot)
Volume (bars / second y-axis)
Highlight suspicious spikes with red dots/arrows + annotations (“Volume 5x daily average → potential manipulation”).
✅ Strength: very intuitive, shows temporal context.
⚠️ Weakness: doesn’t explain who caused it (just that it happened).

2. Histograms of transaction sizes / times 📊
Best for: showing abnormal distributions of trades.
Examples:
Many transactions at the exact same size (could indicate bot activity).
Many trades clustered at odd hours (e.g., 3–4am in UTC → coordinated botnet).
✅ Strength: statistical red flag, great supporting evidence.
⚠️ Weakness: harder for a general audience to immediately understand.

3. Wallet-level heatmaps / flows 🔥
Best for: showing concentration of trading power.
Examples:
If a single wallet dominates volume.
If many wallets funnel into one destination (could be a scam / rug pull).
✅ Strength: exposes centralization / collusion.
⚠️ Weakness: more complex to implement (you need address-level data, not just price/volume).

4. Annotated charts (combo approach) 📝
Best for: a polished report or essay.
Example:
Start with a time series (price & volume).
Mark anomalies (red arrows).
Supplement with a histogram (showing unusual transaction size distribution).
If possible, add wallet concentration chart (heatmap / pie chart of top wallets).
✅ Strength: balanced, tells a clear story.
⚠️ Weakness: requires combining multiple plots → more coding time.
